In [1]:
from symMaps.__init__ import *
from symMaps.lpSys import _adjF
_numtypes = (int,float,np.generic)

# Documentation for `ModelShell` 

## 1. A brief overview

The class `symMaps.ModelShell` builds on `symMaps.LPSys` class to create a platform for defining linear programming models. The model structure is specified using the `symMaps.LPSys` class and the class `ModelShell` simply defines additional routines for 
1. passing the compiled model to a solver (`scipy.optimize.linprog`),
2. unloading the solution to a database, and
3. create routines for running shocks (changes in parameter-values/structure) in the model and reporting it.

An instance of the `symMapsl.LPSys`is added as attribute `self.sys`. The `pyDbs.SimpleDB` database instance tied to the `symMaps.LPSys` instance is also the main database for the `ModelShell` instance; the property `self.db` returns the `self.sys.db` directly.

Main methods:
* `self.solve(**kwargs)`: Passes the compiled linear programming system from `self.sys.out`to the solver `scipy.optimize.linprog`. The `**kwargs` are also passed to the solver, with default solver options stored as the attribute `self.solOptions`. The default option is to use the solver `method = highs`.
* `self.postSolve(sol, **kwargs)`: The `sol` argument here is the solution from the `self.solve` statement, and the `**kwargs` is included to allow for simple extensions for classes building on this. This returns the solution dictionary from calling `self.sys.unloadSol(sol)` (see `docs_LPSys.ipynb`).
* `self.postSolveToDB(sol, **kwargs)`: Adds the solution from `self.postSolve` to the main database in `self.sys.db`.
* `self.lazyLoop(grids, idxLoop, **kwargs)`: This is a simple routine for updating parameter values on a grid and reporting the solution. The `idxLoop` is a `pd.Index` of the loop. The routine goes through:
    1. Update database symbols from `grids`. This can either be a `pd.Series` or an iterator populated by `pd.Series` instances. The structure of the `pd.Series` should be structured in the following way: The name of the series corresponds to a variable in `self.db` and the index should be defined over `idxLoop` and the original domain as stored in `self.db`.
    3. Recompile model calling `self.compile()` method. This assumes that the model has such a method implemented that recompiles the arguments in `self.sys`.
    4. Solve model,
    5. save post-solve routine to dictionary.

    The `self.lazyLoop` returns a dictionary of dictionaries for each element in the grid. Each dictionary is the solution dictionary with keys = symbol names and values = pd.Series representations of the solution.
* `self.lazyLoopAsDFs(grids, idxLoop, **kwargs)`: Collect solutions from `self.lazyLoop` in dataframes with columns representing the grid we loop through.

The `ModelShell` class is an incomplete model in the sense that it does not include the structure of any model. Specifically, as mentioned in the short description above, it requires that we have a defined instance of the class `symMaps.LPSys`. Also, for the loop methods to work, we also want to define `self.compile` methods that actually defines the structures of the `symMaps.LPSys`. 

## 2. Example of implementation

To understand how the `ModelShell` works, we showcase one example that builds a basic electricity system model (`MBasic`). 

In [2]:
import os
from symMaps.mBasic import MBasic
from pyDbs import ExcelSymbolLoader

### A. Data

The model is a very simple power system model that is defined over four indices (parentheses indicate symbols used in plain math below).
* ```idxGen``` ($i$): Set of electricity generators.
* ```idxF``` ($f$): Set of fuel types that generators may use.
* ```idxEm``` ($em$): Set of emission types included in the model.
* ```idxCons``` ($ic$): Set of consumers in the power market.


The model requires the following data to run:
* ```pFuel(idxF)``` ($p_f$): Fuel price. Default unit: €/GJ. 
* ```uEm(idxF, idxEm)``` ($\phi_{f,em}$): Emission intensity. Default unit: Ton emission/GJ fuel input. 
* ```VOM(idxGen)``` ($c_i^{oth}$): variable operating and maintenance costs. Default unit: €/GJ output.
* ```uFuel(idxF, idxGen)``` ($\mu_{f,i}$): Fuelmix. Default unit: GJ input/GJ output.
* ```taxEm(idxEm)``` ($\tau_{em}$): Tax on emissions. Default unit: €/ton emission output.
* ```genCap(idxGen)``` ($q_i$): Generating capacity. Default unit: GJ/hour.
* ```mwp(idxCons)``` ($u_{ic}$): Marginal willingness to pay for consumers. Default unit: €/GJ. 
* ```load(idxCons)``` ($L_{ic}$): Total demand for consumers if the price does not exceed their marginal willingness to pay. Default unit: GJ.

### B. Model formulation

The model solves for vectors of electricity generation ($E_i$ = `generation`) and demand levels ($D_{ic}$ = `demand`) by solving:
$$\begin{align}
    &\max\mbox{ }\sum_{ic}u_{ic} D_{ic} - \sum_{i} c_i E_i \\
    &\text{s.t. }\sum_{ic} D_{ic} = \sum_i E_i \\
    & E_i\in[0,q_i], \quad \text{and} \quad D_{ic} \in[0, L_{ic}],
\end{align}$$
and where the marginal costs of generation, $E_i$, are computed from:
$$\begin{align}
    c_i = c_i^{oth} + \sum_f \mu_{f,i} (p_f + \sum_{em} \phi_{f,em}\cdot\tau_{em})
\end{align}$$

The excel file `EX_MBasic.xlsx` contains an example of the relevant data outlined above (except for values for the endogenous variables $E_i, D_{ic}$ that we are going to solve for). This reads in the data and initializes a model instance that we can use to test out various methods:

In [3]:
testData = os.path.join(os.getcwd(), 'data','EX_MBasic.xlsx')
data = ExcelSymbolLoader(testData)() # read data in a dict 
m = MBasic() # initialize model
[m.db.__setitem__(k, v) for k,v in data.items() if k!='__meta__']; # add data to model database

In this model calss, the `self.compile` method that sets up the structure of the model goes through three steps; we go through them in turn here.

####  *i)* Fundamentals - `self.compileMaps()`

The `self.compileMaps` starts by declaring domains of variables (`self.initArgs_v`), equality constraints (`self.initArgs_eq`), and inequality constraints (`self.initArgs_ub`). In this model, we have:
* Two variables: `generation` ($E_i$) defined over the index `idxGen` ($i$).
* One constraint: `equilibrium` which is a scalar constraint (index `None`).

Note that we include the auxiliary calculation for marginal costs $c_i$ outside the linear programming system; we revisit this below.

With this structure in place, the `self.compileMaps` method then applies `self.sys.compileMaps` which establishes the global indices for variables and constraints: 

In [4]:
m.compileMaps()

####  *ii)* Auxiliary calculations - `self.updateAux`

In the next step, we compute auxiliary variables. For this model, that means adding marginal costs `mc` to the `self.db` (following equation for $c_i$ above). 

In [5]:
m.updateAux()
m.db['mc'].v.head()

idxGen
biomass    14.00
coal       22.66
natgas     27.84
nuclear     2.15
solar       1.00
dtype: float64

We implement this as a standard method in the class because we expect many scenarios likely involve recomputing this object. 

####  *iii)* Specify parameters for `LPSys` - `self.compileParams`

We specify the parameters for the `LPSys` method going through (1) adding coefficients on variables (`c`, `u`, `l`), (2) equality constraints (`A_eq`, `b_eq`), and (3) inequality constraints (`A_ub`, `b_ub`).

**Coefficients on variables:**

In our model, we have two variables to solve for `generation` and `demand`. We specify the coefficients (`c`, `l`, `u`) for each of the variables using methods with syntax `f'self.initArgsV_{k}'` with `k` representing the variable name (from `self.sys.v`). Specifically, we call the following:
```python
    [getattr(self, f'initArgsV_{k}')() for k in self.sys.v]; # specify c,l,u for all variables
```
Thus, if we extend the model with a new variable, we add it to `self.sys.v` (specify its domains) and add a method `self.initArgsV_{k}` that specifies the costs, lower bound, and upper bound. Given this, the `self.compile` method automatically includes this.

For the `generation` variable, for instance, we add the following:

```python
def initArgsV_generation(self):
	self.sys.lp['c'][('mc', 'generation')] = self.db('mc') 
	self.sys.lp['u'][('genCap', 'generation')] = self.db('genCap') 
```
We do not specify a lower bound, as the default is simply zero here.

**Equality constraints:**

Variational constraints (equality/inequality) are added in a similar manner as the coefficients on parameters:for equality constraints, we iterate through methods following a specific syntax `f'self.initArgsEq_{k}'`where `k` represents the specific constraint (from `self.sys.eq`). For inequality constraints, we simply replace `eq` with `ub`.

In this model, we have a single variational constraint in this model, namely the `equilibrium` one. Here we simply add:
```python
def initArgsEq_equilibrium(self):
	self.sys.lazyA('eq2Gen', series = 1,  v = 'generation', constr = 'equilibrium',attr='eq')
	self.sys.lazyA('eq2Dem', series = -1, v = 'demand', constr = 'equilibrium',attr='eq')
```
We rely on the `lazyA` method for adding the coefficients on `generation` and `demand`; this broadcasts the scalars (1 and -1, respectively) to the full domain of the constraint/variable. If the constraint requires a constant different from zero, we would have added it here to the `self.sys.lp['b_eq']` structure. Without adding any, the compiler defaults to zero. 

The `self.compileParams` sets up the structure and returns the `self.sys.out` that can be passed to the linear programming solver: 

In [6]:
m.compileParams()

{'c': array([  1.  ,   1.  ,   2.15,  14.  ,  27.84,  22.66, -30.  , -25.  ]),
 'A_ub': <COOrdinate sparse array of dtype 'float64'
 	with 0 stored elements and shape (0, 8)>,
 'b_ub': array([], dtype=float64),
 'A_eq': <COOrdinate sparse array of dtype 'int64'
 	with 8 stored elements and shape (1, 8)>,
 'b_eq': array([0.]),
 'bounds': array([[   0.,  720.],
        [   0.,  360.],
        [   0., 3600.],
        [   0.,  180.],
        [   0., 1440.],
        [   0., 2880.],
        [   0., 2000.],
        [   0., 7000.]])}

### C. Post solution routine - `self.postSolve`

The steps above can be summarized by the following two calls:

In [8]:
m.compile()
sol = m.solve()

The `sol` is the solution object returned from `scipy.optimize.linprog`. Note that the `self.solve` includes an assertion that `sol['status'] == 0`, i.e. it raises an error if the optimization is not successful. For this model, we implement the following small `postSolve` routine:
```python
def postSolve(self, sol, **kwargs):
	solDict = super().postSolve(sol)
	solDict['surplus'] = -sol['fun']
	solDict['fuelCons'] = fuelConsumption(solDict['generation'], self.db('uFuel'))
	solDict['emissions'] = emissionsFuel(solDict['fuelCons'], self.db('uEm'))
	return solDict
```

Recall that the parent class (`ModelShell`) simply returns a dictionary with solution variables (`generation`, `demand`) and dual variables for variational constrains and domain constraints. On top of this, we add three variables: (1) A surplus, (2) fuel consumption, and (3) emissions. The `fuelConsumption` and `emissionsFuel` statements refer to locally defined functions. The solution from this statement:

In [11]:
solDict = m.postSolve(sol)
solDict.keys()

dict_keys(['generation', 'demand', 'λeq_equilibrium', 'λl_generation', 'λl_demand', 'λu_generation', 'λu_demand', 'surplus', 'fuelCons', 'emissions'])

If we want, we can add this directly to our database in `self.db` by calling the `self.postSolveToDB` instead; this draws directly on our customized `self.postSolve` routine and so, we automatically get the new variables with us here:

In [14]:
m.postSolveToDB(sol) # add solution to internal daatbase instead
m.db('generation').head() # look at solution for a bit

idxGen
wind        720.0
solar       360.0
nuclear    3600.0
biomass     180.0
natgas        0.0
Name: generation, dtype: float64

## 3. `lazyLoop` shocks

Finally, we showcase the `self.lazyLoop` method that is also added to the `ModelShell` class. The loop goes through four steps: (1) Update database values from specified grids, (2) compile model again from scratch, (3) solve, (4) save post-solve routine to dict. 

We refer to this as a "lazy" loop, because it allows us to do a lot with small bits of code, but it spends unnecessary time recompiling the entire model from scratch; recall that once the model is compiled, the `self.sys.out` dictionary stores the relevant arguments that we need to pass to a solver. Often, the shock/simulation we have in mind can be carried out by simply adjusting selected elements in `self.sys.out` and then resolving. 

In [22]:
data.keys()

dict_keys(['idxGen', 'idxF', 'idxEm', 'idxCons', 'pFuel', 'uEm', 'VOM', 'uFuel', 'taxEm', 'genCap', 'mwp', 'load', '__meta__'])

As an example, let us say that we want to see how the model depends on the cost of emissions. This is captured by a parameter `taxEm` which enters the model through the marginal cost of generators. We set up a loop with 10 different values, from 0 to 100 (unit here is €/ton CO2), solve the model for each level and extract the solution in a dictionary with dataframes:

*Set up grid:*

In [31]:
name = 'taxEm'
idxLoop = pd.Index(range(10), name = 'loop')
v0, vT = pd.Series(0, index = m.db(name).index), pd.Series(100, index = m.db(name).index)
grid = adjMultiIndex.addGrid(v0, vT, idxLoop,name)
grid.head()

loop  idxEm
0     CO2       0.000000
1     CO2      11.111111
2     CO2      22.222222
3     CO2      33.333333
4     CO2      44.444444
Name: taxEm, dtype: float64

*Run loop and extract solution in dictionary of dataframes:*

In [36]:
solDFs = m.lazyLoopAsDFs(grid, idxLoop)

*Look at optimal generation as a function of the tax:*

In [43]:
solDFs['generation'].head()

loop,0,1,2,3,4,5,6,7,8,9
idxGen,,,,,,,,,,
wind,720.0,720.0,720.0,720.0,720.0,720.0,720.0,720.0,720.0,720.0
solar,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0
nuclear,3600.0,3600.0,3600.0,3600.0,3600.0,3600.0,3600.0,3600.0,3600.0,3600.0
biomass,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0
natgas,1260.0,1260.0,1260.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
